In [1]:
pip install konlpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tqdm import tqdm

In [2]:
from datetime import datetime

In [3]:
import sys
import os
import pandas as pd

sys.path.append('/app/EZPZ_Project/modules/torchmodules') # 토치 모델 뉴스 요약 및 감정평가 가져오기

In [4]:
import pymysql
import cryptography
import sql_connection as sc #mysql connection

In [5]:
from privates.ezpz_db import * #db연동 정보

In [6]:
from tokenizer import get_keyword_nnp, get_keyword_nng #토크나이저 호출 nnp는 

In [7]:
#토치 모듈 키워드추출 토크나이저와 t5 base sum 리뷰 요약
#import tokenizer 토크나이저는 일단 나중에
from service_models import ServiceModels #t5model

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data_check= ServiceModels() #모델 서빙 모듈 객체

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1. 분기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_quater() return 분기별 리뷰 리스트/ 03(1분기) 04(2분기) 05(3분기) 06(4분기)
2. 반기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_half() return 반기별 리뷰 리스트/ 01(전반기) 02(하반기)
3. 모든 기간 데이터로 정보 가져오는 함수 data_not_split 00: 모든 기간 return 모든기간 리뷰 리스트(해당 회사 전부 리뷰 요약)
4. 키워드 추출하는 함수 (데이터 나눠서 크롤링하는 함수에 넣어줌) get_keyword(review_list) 이지가 리스트로 받기도 가능하다 했으니
5. 리뷰 내용 요약 함수(매개변수: 분기/반기 별로 나눠진 리뷰 리스트) (크롤링하는 함수에 넣어줌)
6. 이모든걸 실행해주는 주요 함수: review_sum(all_comp_name):


In [9]:
def get_year_list(split_data): #분기/반기별 분할된 데이터를 그룹핑할 기준인 연도리스트 얻음
    i = 0
    year_list = [] #년도 개수 체크
    for item in range(len(split_data)):
        year = split_data[item][3][:4]  # YYYY 형식의 연도 추출
        if year not in year_list:
            year_list.append(year) #한 분기/반기에 포함돼있는 연도 리스트 [2023 ,1992...]
    return year_list #이걸로 그루핑 하고 uid 얻을때도 쓰고



In [10]:
def no_data_split(): #스플릿안된 데이터 넘겨주는 함수

    sql = ' SELECT comp_uid, review_cont, review_rate, review_date '
    sql += ' FROM comp_review '
    no_split_data = sc.conn_and_exec(sql)
    return no_split_data


In [11]:
def data_split_by_half():# 반기별로 분활하는 함수(밑에 sql문 참고)
    half_data = [] #반기들을 모아놓은 리스트
    sql = ' SELECT comp_uid, review_cont, review_rate, review_date '
    sql += ' FROM comp_review '
    sql += " WHERE review_date LIKE '____01' OR review_date LIKE '____02' OR review_date LIKE '____03' OR review_date LIKE '____04' OR review_date LIKE '____05' OR review_date LIKE '____06'; "

    half_one_data =sc.conn_and_exec(sql)

    sql = ' SELECT comp_uid, review_cont, review_rate, review_date '
    sql += ' FROM comp_review '
    sql += " WHERE review_date LIKE '____07' OR review_date LIKE '____08' OR review_date LIKE '____09' OR review_date LIKE '____10' OR review_date LIKE '____11' OR review_date LIKE '____12'; "
    
    half_two_data = sc.conn_and_exec(sql)
    half_data.append(half_one_data)
    half_data.append(half_two_data)
    return half_data

In [12]:

def data_split_by_quater(): #회사이름과 시간분할 기준 이제 회사별로 이걸 해줘야지

        quater_data = [] #분기들을 모아놓은 리스트
        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += f" WHERE review_date LIKE '____01' OR review_date LIKE '____02' OR review_date LIKE '____03'; " #1분기의 리뷰 데이터

        #1분기 데이터 연도 구분 x
        quater_one_data = sc.conn_and_exec(sql)



        sql = ' SELECT comp_uid,review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____04' OR review_date LIKE '____05' OR review_date LIKE '____06'; "

        #2분기 데이터 연도 구분 x
        quater_two_data = sc.conn_and_exec(sql)

        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____07' OR review_date LIKE '____08' OR review_date LIKE '____09'; "

        #3분기 데이터 연도 구분 x
        quater_three_data = sc.conn_and_exec(sql)

        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____10' OR review_date LIKE '____11' OR review_date LIKE '____12'; "

        #4분기 데이터 연도 구분 x
        quater_four_data = sc.conn_and_exec(sql)

        quater_data.append(quater_one_data)
        quater_data.append(quater_two_data)
        quater_data.append(quater_three_data)
        quater_data.append(quater_four_data)
        return quater_data

    #data[0] 리뷰내용/ data[1] 리뷰감성평가/ data[2] 리뷰평점/ data[3] 리뷰날짜









In [ ]:
crawling_all

In [ ]:
no_split_data = no_data_split()
year_list = get_year_list(no_split_data)
comp_uid_review_sum = sum_review_dict(no_split_data,year_list,'00')

In [ ]:
comp_uid_review_sum

In [31]:
def get_keyword_list(comp_uid_review_sum): #연도 및 분/반기 요약돼있는 dict 받아서 키워드 분할 counter도 써서 빈도수 나오면(1)을 옆에 _빈도수 써줘서 for문돌려서 #추가 하나키워드 나올때마다
     #keyword_hashtag_list = []
     #for key,value in sum_dict.items():
     #comp_review_list=[]
     year_keyword_list = []
     keyword_hashtag_list = []
     for uid_reviews in comp_uid_review_sum: #각연도의 uid와 uid별 review들을 묶어둔 이차원 리스트
          for comp_uid_review in uid_reviews: # uid 하나와 리뷰로 이루어진 1차원리스트

               review_keyword = get_keyword_nng([comp_uid_review[2]],'review') #[2]이 각회사의 요약된 리뷰
               #print(review_keyword) #각 회사별 연도,분.반기 구분된 요약 내용의 키워드 추출 여기서 키워드 추출해서 회사별로 #과 _빈도수 붙여주기
               #연도별로 회사마다 나온 키워드를 저장
               year_keyword_list.append(review_keyword)

     for comp_keyword in year_keyword_list: #comp_keyword 는 한회사의 키워드로 ('온보딩',1) 꼴로 투플을 순회
          keyword_hashtag = '' # 키워드를 해시태그로 저장할 문장
          for keyword in comp_keyword:
               keyword_hashtag += keyword[0]+'_'+ str(keyword[1])+'#'
          keyword_hashtag =keyword_hashtag[:-1] #마지막 #은 제거
          keyword_hashtag_list.append(keyword_hashtag)


     return keyword_hashtag_list
     return year_keyword_list

          #temp_list.append(split_review)
          #return temp_list

In [44]:
get_keyword_list(crawling_all)

[]

In [21]:
#quater_one_data에서 요약한 애들 리스트를 합치기전에 받아서 코딩을 해야겠네 그러려면 sum_review_dict에서 일부코드만 따와야겠네
def get_avg_rate_list(split_data,year_list): #comp_uid_review_sum은 안쓴 확률 높음 평점항목이 없어서
    year_rate_average_list = [] #별점을 다 모아오는 리스트
    rate_sum = 0 #각회사의 연도 분/반기별 평점 합계 담아줄 변수 나중에 개수로 나눠 평균 구해줄 예정
    rate_count = 0 # 각 회사의 평점  개수
    uid_rate_year_list = []
    for year in year_list: #2023
        for item in range(len(split_data)): #테이블에서 가져온 데이터중 분기할 데이터들만 가져오기
                if split_data[item][3][:4] == year:
                    #split_data_dict[year].append(split_data[item][0]) #comp_uid를 딕셔너리에 넣어줌

                    uid_rate_year_list.append([year,split_data[item][0],split_data[item][2]]) #연도별(year) 회사uid와 리뷰 평점

                    """    if item+1 < len(split_data) and split_data[item][0] == split_data[item+1][0]: #순서대로 리뷰가 정렬되므로 이렇게 평점 더해줘도 됨 스플릿데이터 comp도 순서대로 1,2~
                             rate_sum += split_data[item][3] #그 회사의 평점 합을 더해줌
                             rate_count += 1 # 평점 개수 +1

                        else: #comp_uid 가 달라진경우
                              rate_sum+= split_data[item][0] #마지막으로 같았던 comp_uid의 평점 넣어줌
                              rate_count +=1
                              print(split_data[item][0])
                              year_rate_average_list.append([split_data[item][0],year,rate_sum/rate_count])"""

    #각 회사별 연도별 분/반기별 평점 리스트 갖고 오는 알고리즘

    result_dict = {}

    for item in uid_rate_year_list:
        year = item[0]
        key = (year, item[1]) # (연도, 두 번째 인덱스 값)을 키로 사용

        if key in result_dict: #딕셔너리안에 키가 없으면 항목 넣어주고 키에 맞춰서 append 평점 
            result_dict[key].append(item[2])
        else:
            result_dict[key] = [item[2]]


    for key in result_dict:
        values = result_dict[key]
        average_value = sum(values) / len(values)
        year_rate_average_list.append([key[0], key[1], round(average_value,1)])
    return year_rate_average_list#year_rate_average_list


In [107]:
#1. 연도별로 seperate 2. 연도별 seperate 된 애들중 같은 분기끼리는 전부 concat 요약 모델 for문
def sum_review_dict(split_data,year_list,split_time): #분기/반기별로 모아놓은 list quater_data[0]이 1분기 [1] [2] [3] 이 각각 분기 [0][0]은 그 분기정보중 한 row [0][0][0]은cont 하나
    split_data_dict ={}
    new_value_list = []
    #combined_sum_review_list = []
    i = 0 #얼마나 왔는지 체크용
    for year in year_list:
        split_data_dict[year] = [] #연도별로

    for year in year_list:
        for item in range(len(split_data)):
            if split_data[item][3][:4] == year:
                #split_data_dict[year].append(split_data[item][0]) #comp_uid를 딕셔너리에 넣어줌
                split_data_dict[year].append([split_data[item][0],split_data[item][1]]) #분기/반기 리뷰를 또 연도별로 구분해서 담아줌 uid를 어떻게 구분해서 join하지
                #일단 현재 튜플로 comp_uid와 리뷰내용을 묶어놓은 상태 여기서 어떻게 분리해서 요약따로하지

    #이제 이 내용 담긴 딕셔너리를 key별(연도)로 concat 하면 dict안에 연도별 분기별로 구분된 요약내용이 담긴 딕셔너리 완성
    for key,value in split_data_dict.items(): #딕셔너리 안에 있는 연도별 value들
        #for tp in value: #각 연도별 리스트 안에 있는 튜플들(리뷰와 comp_uid)
        combined_sum_review_list = [] # 매 키가 순회할때마다 리스트 비워줌
        merged_reviews = []
        current_key = None
        current_text = ""

        for key, text in value:
            if key == current_key:
                current_text += " " + text
            else:
                if current_key is not None:
                    merged_reviews.append([current_key, current_text])
                current_key = key
                current_text = text

        # 마지막 데이터 처리
        if current_key is not None:
            merged_reviews.append([current_key, current_text]) # 각 연도별 리뷰같은회사끼리 한꺼번에 묶어준 튜플

        value =merged_reviews #각 연도별 같은 회사 묶어준 튜플

        for combined_review_index in tqdm(value): #각 tp가 생각해보니까 곧 리뷰 합친거잖아
            #combined_review = ''.join(tp[1]) #연도별 리뷰들을 한꺼번에 묶어줌
            combined_sum_review = data_check.get_summary(combined_review_index[1],'reviews_long') #연도별 분/반기별 묶인 리뷰 요약
            combined_sum_review_list.append(combined_sum_review)
            if combined_review_index[0]>3: #테스트용 코드
                break #테스트용으로 회사 100개만 확인
        print(i)
        i+=1

        new_value_temp = [[split_time,v[0], combined_sum_review_list[i]] for i, v in enumerate(value) if v[0]<3 ] #회사별 요약해준 값과 uid를 리스트로 묶어 다시 value 리스트에 넣어줌
        new_value_list.append(new_value_temp) #각 연도별 회사들의 리뷰 요약 을 리스트의 인덱스로 가지는 리스트 [[(1,~),(2,~)],[]]
        print(new_value_list)
    return new_value_list #일단은 comp_uid와 그에 맞는 요약된 리뷰를 담아주기 나중에 이 리스트들 한번에 합쳐주기

        #이것도 테스트용이라 value가 t[0]<10 까지만 해놨어 나중에 저 if절 빼면됨
    #for key, value in split_data_dict.items():
    #    split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌


    """for key,value in split_data_dict.items():
        if len(new_value_list) == 0: # 원래는 이코드 넣을 필요없는데 테스트코드라 딕셔너리 year에 포함안된회사(10개라) 있어서 길이가 달라짐 그러므로 테스트용
            break
        split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌

    return split_data_dict # 성공!!! 연도별 분/반기별 회사별 요약본 만들기 이제는 분/반기 그루핑하면 됨"""



In [108]:
#함수들 다 실행해서 db까지 저장해주는 main function
"""
실행 메커니즘
1.리뷰 긁어올 회사를 받아옴
2.분기별/반기별 데이터를 분할
3.각 분할된 데이터에 있는 연도들을 리스트로 뽑아옴
4.그 연도와 분/반기를 보내 내용 요약 딕셔너리로 만들어줌
5.회사 아이디에 이어붙여
"""

def exec_all():

    #전체 데이터
    all_crawl_data = []
    no_split_data = no_data_split()
    year_list = get_year_list(no_split_data)
    comp_uid_review_sum = sum_review_dict(no_split_data,year_list,'00')
    keyword_list = get_keyword_list(comp_uid_review_sum)
    avg_rate_list = get_avg_rate_list(no_split_data,year_list)
    all_crawl_data.append((year_list,comp_uid_review_sum,keyword_list,avg_rate_list))

    #반기
    half_crawl_data = []
    half_data = data_split_by_half()
    #상반기
    year_list_half_one = get_year_list(half_data[0])
    comp_uid_review_sum_half_one = sum_review_dict(half_data[0],year_list_half_one,'01')
    keywoord_list_half_one = get_keyword_list(comp_uid_review_sum_half_one)
    avg_rate_list_half_one = get_avg_rate_list(half_data[0],year_list_half_one)

    #하반기
    year_list_half_two = get_year_list(half_data[1])
    comp_uid_review_sum_half_two = sum_review_dict(half_data[1],year_list_half_two,'02')
    keywoord_list_half_two = get_keyword_list(comp_uid_review_sum_half_two)
    avg_rate_list_half_two = get_avg_rate_list(half_data[1],year_list_half_two)

    half_crawl_data.append([year_list_half_one,comp_uid_review_sum_half_one,keywoord_list_half_one,avg_rate_list_half_one])
    half_crawl_data.append([year_list_half_two,comp_uid_review_sum_half_two,keywoord_list_half_two,avg_rate_list_half_two])
    ###
    #분기
    ###
    quater_crawl_data = []

    #1분기
    quater_data = data_split_by_quater() #분기별 데이터 분할 1,2,3,4 #나중에는 그냥 분기 4개 for문에 다넣어버려서 코드 길이 줄이자

    year_list_quater_one = get_year_list(quater_data[0]) #1분기 "03"
    comp_uid_review_sum_quater_one = sum_review_dict(quater_data[0],year_list_quater_one,'03') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_quater_one = get_keyword_list(comp_uid_review_sum_quater_one) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_quater_one = get_avg_rate_list(quater_data[0],year_list_quater_one) #얘만 지금 한분기 전체 데이터가 다나옴 나머지는 잘랐어서 실제 실행할때는 테스트용 코드 전부 제거

    #2분기
    year_list_quater_two = get_year_list(quater_data[1]) #1분기 "03"
    comp_uid_review_sum_quater_two = sum_review_dict(quater_data[1],year_list_quater_two,'04') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_quater_two = get_keyword_list(comp_uid_review_sum_quater_two) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_quater_two = get_avg_rate_list(quater_data[1],year_list_quater_two)

    #3분기
    year_list_quater_three = get_year_list(quater_data[2]) #1분기 "03"
    comp_uid_review_sum_quater_three = sum_review_dict(quater_data[2],year_list_quater_three,'05') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_quater_three = get_keyword_list(comp_uid_review_sum_quater_three) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_quater_three = get_avg_rate_list(quater_data[2],year_list_quater_three)
    #4분기
    year_list_quater_four = get_year_list(quater_data[3]) #1분기 "03"
    comp_uid_review_sum_quater_four = sum_review_dict(quater_data[3],year_list_quater_four,'06') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_quater_four = get_keyword_list(comp_uid_review_sum_quater_four) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_quater_four = get_avg_rate_list(quater_data[3],year_list_quater_four)

    quater_crawl_data.append([year_list_quater_one,comp_uid_review_sum_quater_one,keyword_list_quater_one,avg_rate_list_quater_one])
    quater_crawl_data.append([year_list_quater_two,comp_uid_review_sum_quater_two,keyword_list_quater_two,avg_rate_list_quater_two])
    quater_crawl_data.append([year_list_quater_three,comp_uid_review_sum_quater_three,keyword_list_quater_three,avg_rate_list_quater_three])
    quater_crawl_data.append([year_list_quater_four,comp_uid_review_sum_quater_four,keyword_list_quater_four,avg_rate_list_quater_four])

    merged_list = merge_lists(all_crawl_data,half_crawl_data,quater_crawl_data) #이 리스트에 각각 [0] 전체에 정보 [1] 반기 [2] 분기

    save_db(merged_list) #연도순대로 저장된게 mergedlist니까 연도 하나당 리뷰정보들 리스트 하나 담긴 회사랑 다 for문돌려서 가져오면됨
    #return quater_crawl_data#all_crawl_data,half_crawl_data,quater_crawl_data



"""


    #분기 딕셔너리에
    return comp_uid_review_sum_one,keyword_list_one,avg_rate_list_one
    #return(avg_rate_list)
    return year_list

    #quater_two_data = data_split_by_quater(all_comp_name,2) #분기별 데이터 분할 1,2,3,4
    #quater_three_data = data_split_by_quater(all_comp_name,3) #분기별 데이터 분할 1,2,3,4
    #quater_four_data = data_split_by_quater(all_comp_name,4) #분기별 데이터 분할 1,2,3,4



    #sum_review(quater_one_data,quater_two_data,quater_three_data,quater_four_data) #분기,반기 일단 분기만 했는데 내일 요거 토대로 반기도 넣어야됨

    #db_save(sum_review) #각각의 얻은 디비에 넣을 값들 바탕으로 매개변수 받아서 db세이브"""


'\n\n\n    #분기 딕셔너리에\n    return comp_uid_review_sum_one,keyword_list_one,avg_rate_list_one\n    #return(avg_rate_list)\n    return year_list\n\n    #quater_two_data = data_split_by_quater(all_comp_name,2) #분기별 데이터 분할 1,2,3,4\n    #quater_three_data = data_split_by_quater(all_comp_name,3) #분기별 데이터 분할 1,2,3,4\n    #quater_four_data = data_split_by_quater(all_comp_name,4) #분기별 데이터 분할 1,2,3,4\n\n\n\n    #sum_review(quater_one_data,quater_two_data,quater_three_data,quater_four_data) #분기,반기 일단 분기만 했는데 내일 요거 토대로 반기도 넣어야됨\n\n    #db_save(sum_review) #각각의 얻은 디비에 넣을 값들 바탕으로 매개변수 받아서 db세이브'

In [ ]:
crawling_all = exec_all()

In [ ]:
exec_all() #이게 실행 키

In [ ]:
crawling_all[0][0][1] #첫번째 인덱스(전체/반/분기)로 분기 접근 두번째 인덱스로는 각각의 분기에 디테일 세번째 인덱스는 각각의 필요 정보접근 각자의 인덱스번호별로 디비에 저장해주면 됨

In [171]:
crawling_all[0][0][2]

['스타트업_1#회사_1#직원_1#발전_1#성장_1',
 '근무_4#시간_2#생활_1#환경_1#도입_1#직원_1#개발_1#편의_1#주간_1#업무_1',
 '재택근무_1#일자리_1#정규직_1#과정_1#시험_1#기회_1',
 '의지_2#수준_2#직원_1#교육_1#육성_1#검증_1#상사_1#기회_1#직장_1#현황_1',
 '온보딩_1#기초_1#회사_1#적응_1#환경_1#장점_1',
 '워라밸_1#직원_1#연령대_1#친구_1',
 '스타트업_1#회사_1#성장_1#목표_1#액션_1#플랜_1#구체_1#단계_1#구성원_1#제안_1',
 '업무_2#처리_2#시간_2#부족_2#수평_1#조직_1#문화_1#야근_1#강요_1#연차_1',
 '회사_2#경험자_1#재택근무_1#시스템_1#자유_1#분위기_1#직원_1#권한_1#이직_1#이력_1',
 '체계_2#스타트업_1#남녀_1#성비_1#균형_1',
 '회사_2#직원_1#사람_1#업무_1#전반_1#교육_1#체계_1#연봉_1#인상_1#진행_1',
 '경영진_1#능력_1#인사_1#사람_1#회사_1#생각_1',
 '직원_4#회사_2#속도_2#경영진_1#업무_1#실력_1#인맥_1#자본_1#우수_1#대표_1',
 '연장_3#책임_3#월급_3#권한_3#동시_3#근무_3#시간_3',
 '기업_2#인사_1#신입_1#지원_1#인턴_1#추천_1',
 '컨설팅_1#성장_1',
 '업무_1#강도_1',
 '인사_1#전문_1#컨설팅사_1#역량_1#매니지먼트_1#시니어_1#컨설턴트_1#이하_1#인력_1#부족_1']

In [94]:
quater_crawl_data =crawling_all[:]

In [69]:
all_crawl_data = crawling_all[0][:]
half_crawl_data = crawling_all[1][:]
quater_crawl_data = crawling_all[2][:]

In [99]:
exec_list = merge_lists(quater_crawl_data)#all_crawl_data,half_crawl_data,quater_crawl_data)

In [101]:
def merge_lists(all_crawl_data,half_crawl_data,quater_crawl_data): #키워드와 회사 먼저 머지하고 year랑 머지해야겠네
    merged_list = []
    """for i in range(len(list1)):
        merged_list.append((list1[i], list2[i]))
    return merged_list"""
    #전체 내용
    i = 0
    for reviews in all_crawl_data[0][1]:
            #for years_reviews_kw in split_time[0]: #연도별 회사리뷰를 순회 [0][0][0]:years [0][0][1]:reviews [0][0][2] keyword
            for review in reviews:
                review.append(all_crawl_data[0][2][i]) #리뷰와 키워드 합치기
                i += 1

    i = 0
    for split in half_crawl_data: #반기 데이터 리뷰와 키워드 머지
        i=0
        for reviews in split[1]:
               for review in reviews:
                      review.append(split[2][i])
                      i+=1
    i = 0
    for split in quater_crawl_data: #분기데이터 리뷰와 키워드 머지
        i = 0
        for reviews in split[1]:
               for review in reviews:
                      review.append(split[2][i])
                      i+=1


    return all_crawl_data,half_crawl_data,quater_crawl_data


In [126]:
save_db(exec_list)

In [125]:
def save_db(merged_list): #연도순대로 저장된게 mergedlist니까 연도 하나당 리뷰정보들 리스트 하나 담긴 회사랑 다 for문돌려서 가져오면됨
     #그 때 그때 정보를 받아오면 분기번호 따라서 연결해주기 하고 데이터 전부 테이블에 저장 인덱스번호별로 일단 avg리스트는 5 넣어놓고 짜논코드는 주석처리

     ###############

     # 여기서 갖고 올 수 있는 것들 다 갖고오기 uid며 요약 sum_year ,sum_term분기, avg_rate는 일단 1값 넣어두고
     #merged_list 정보 다 뺴오고(comp_uid sum_term,sum_cont,sum_keyword) 리스트 별로 다빼와서 변수 저장후 분기까지 넣어주고 해야됨 (분기도 시계열에 값넣어주자 문자열로)
     # for문으로 리스트 하나하나 딕셔너리로 만들어서 그 딕셔너리를 for문 돌려주면 되잖어 key,value items()로 value는 대신 이중for문 써야되고 한회사마다므로
     ###############
     for split in merged_list: #분기별
          year_list = split[0]
          reviews_list = split[1]

          review_sum_dict = {} #키는 연도 value는 정보 모아둔 리스트해서 넣어줄 딕셔너리

          for i in range(len(year_list)): #연도와 컴프유아이디리뷰키워드들을 꺼내서 딕셔너리로 만들어줌
               year = year_list[i]
               reviews = [review for review in reviews_list[i]]
               review_sum_dict[year] = reviews

          create_date = datetime.today().strftime('%Y%m%d')
          modify_date = datetime.today().strftime('%Y%m%d')

          #평균 별점 넣어주기
          """for rate in merged_list[1][-1]: #평점 컬럼
               if rata[0] == merged_list[0]""" #평점컬럼의 연도와 머지리스트의 연도와 같고 회사가 같으면 포함시켜서 딕셔너리에 다 추가 이거 관련해서는 내일 생각해보자

          for key, values in review_sum_dict.items():
               #review = row['review_'][:1000].replace('"', '').replace("'", '').replace('\\', '').replace('\n', ' ')
               for value in values: #약간 요런느낌? # value가 딱 한회사 리스트
                    sql = 'insert into sum_review '
                    sql += '    (comp_uid, sum_year, sum_term, sum_cont, sum_keyword, avg_rate, create_date, modify_date) '
                    sql += 'values ( '
                    sql += f'   "{value[1]}", "{key}", "{value[0]}", "{value[2]}", "{value[3]}", "5" ' #5 임시로 별점 넣어둠
                    sql += f'    ,"{create_date}", "{modify_date}" '
                    sql += ') '

                    sc.conn_and_exec(sql)
